In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import sys
import time
import shutil
sys.path.insert(0, '..')

from database.dataset import INBreast_Dataset
from metrics.metrics_utils import get_tp_fp_fn_center_patch_criteria
from mc_candidate_proposal.morphology_mc import MorphologyCalcificationDetection
from mc_candidate_proposal.hdog_mc import HDoGCalcificationDetection
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection

from tqdm import tqdm
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
from pathlib import Path

import warnings
import seaborn as sns
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

### Define experiment runers

Morphology

In [6]:
def run_experiment_gsm(threshold, min_distance, area, db, results_path):
    normal_ids = db.get_free_of_selected_lesions_imgs_ids()

    m_results_path = results_path / 'morphology'
    m_results_path.mkdir(exist_ok=True, parents=True)

    rbd_path = m_results_path / 'recounstructed_by_dialation_img'
    shutil.rmtree(rbd_path)
    rbd_path.mkdir(exist_ok=True, parents=True)

    result_det = {}
    res = []
    for idx in tqdm(range(len(db)), total=len(db)):
        case = db[idx]
        image = case['img']
        image_id = case['img_id']
        lesion_mask = case['lesion_mask']
        muscle_mask = case['muscle_mask']
        muscle_mask = case['muscle_mask']

        md = MorphologyCalcificationDetection(
            rbd_path, threshold=threshold, min_distance=min_distance,
            area=area*area, store_intermediate=False, filter_muscle_region=True)

        start = time.time()
        candidate_blobs = md.detect(image, image_id, muscle_mask)
        t = time.time() - start

        tp, fp, fn, ignored_candidates = get_tp_fp_fn_center_patch_criteria(
            candidate_blobs, lesion_mask, None, 14)

        img_res = {
            'img_id': image_id, 'TP': len(tp), 'FP': len(fp), 'FN': len(fn),
            'time': t, 'normal': image_id in normal_ids}

        res.append(img_res)
        result_det[image_id] = {
            'tp': tp.loc[:, ['x', 'y', 'radius']].values,
            'fp': fp.loc[:, ['x', 'y', 'radius']].values,
            'fn': fn.loc[:, ['x', 'y', 'radius']].values}
    results_morphology = pd.DataFrame(res)
    filename = f'md_th-{threshold}_mindist-{min_distance}_area-{area}'
    results_morphology.to_csv(results_path/f'{filename}.csv')
    with open(results_path/f'{filename}.pkl', 'wb') as f:
        pickle.dump(result_det, f)

HDoG

In [8]:
def run_experiment_hdog(dog_parameters, hessian_parameters, db, results_path):
    
    normal_ids = db.get_free_of_selected_lesions_imgs_ids()
    
    hdog_results_path = results_path / 'HDoG'
    hdog_results_path.mkdir(exist_ok=True, parents=True)

    processed_imgs_path = hdog_results_path / 'hdog_images'
    shutil.rmtree(processed_imgs_path)
    processed_imgs_path.mkdir(exist_ok=True, parents=True)

    detections_path = hdog_results_path / 'hdog_detections'
    shutil.rmtree(detections_path)
    detections_path.mkdir(exist_ok=True, parents=True)

    result_det = {}
    res = []

    for idx in tqdm(range(len(db)), total=len(db)):
        case = db[idx]
        image = case['img']
        image_id = case['img_id']
        lesion_mask = case['lesion_mask']
        muscle_mask = case['muscle_mask']

        hdogd = HDoGCalcificationDetection(
            dog_parameters, hessian_parameters, processed_imgs_path, detections_path, filter_muscle_region=True)
        
        start = time.time()
        detections = hdogd.detect(image, image_id, False, False, muscle_mask)
        t = time.time() - start

        tp, fp, fn, _ = get_tp_fp_fn_center_patch_criteria(
            detections, lesion_mask, None, 14)

        img_res = {
            'img_id': image_id, 'TP': len(tp), 'FP': len(fp), 'FN': len(fn),
            'time': t, 'normal': image_id in normal_ids}

        res.append(img_res)
        result_det[image_id] = {
            'tp': tp.loc[:, ['x', 'y', 'radius']].values,
            'fp': fp.loc[:, ['x', 'y', 'radius']].values,
            'fn': fn.loc[:, ['x', 'y', 'radius']].values}

    results_hdog = pd.DataFrame(res)

    ms = dog_parameters['min_sigma']
    Ms = dog_parameters['max_sigma']
    ns = dog_parameters['n_scales']
    ns = dog_parameters['n_scales']
    dth = dog_parameters['dog_blob_th']
    dovp = dog_parameters['dog_overlap']
    dmd = dog_parameters['dog_min_dist']
    filename = f'hdog_ms-{ms}_Ms-{Ms}_ns-{ns}_dth-{dth}_dovp-{dovp}_dmd-{dmd}'
    results_hdog.to_csv(results_path/f'{filename}.csv')
    with open(results_path/f'{filename}.pkl', 'wb') as f:
        pickle.dump(result_det, f)

Hough

In [6]:
def run_experiment_hough(dehazing_params, hough1_params, hough2_params, db, results_path):
    normal_ids = db.get_free_of_selected_lesions_imgs_ids()

    hough_results_path = results_path / 'Hough'
    hough_results_path.mkdir(exist_ok=True, parents=True)

    processed_imgs_path = hough_results_path / 'hough_images'
    shutil.rmtree(processed_imgs_path)
    processed_imgs_path.mkdir(exist_ok=True, parents=True)

    back_ext_radius = 50
    erosion_iter = 20
    erosion_size = 5

    result_det_h1 = {}
    result_det_h2 = {}
    res_h1 = []
    res_h2 = []

    for idx in tqdm(range(len(db)), total=len(db)):
        case = db[idx]
        image = case['img']
        image_id = case['img_id']
        lesion_mask = case['lesion_mask']
        muscle_mask = case['muscle_mask']

        hd = HoughCalcificationDetection(
            dehazing_params, back_ext_radius, processed_imgs_path, hough1_params, hough2_params,
            erosion_iter=erosion_iter, erosion_size=erosion_size, filter_muscle_region=True)

        start = time.time()
        h1_circles, h2_circles = hd.detect(image, image_id,
            load_processed_images=False, hough2=True, muscle_mask=muscle_mask)
        t = time.time() - start

        tp, fp, fn, ignored_candidates = get_tp_fp_fn_center_patch_criteria(h1_circles, lesion_mask, None, 14)
        result_det_h1[image_id] = {
            'tp': tp.loc[:, ['x', 'y', 'radius']].values,
            'fp': fp.loc[:, ['x', 'y', 'radius']].values,
            'fn': fn.loc[:, ['x', 'y', 'radius']].values}
        img_res_h1 = {'img_id': image_id, 'TP': len(tp), 'FP': len(fp), 'FN': len(fn),
            'time': t, 'normal': image_id in normal_ids}
        res_h1.append(img_res_h1)

        tp, fp, fn, ignored_candidates = get_tp_fp_fn_center_patch_criteria(h2_circles, lesion_mask, None, 14)
        result_det_h2[image_id] = {
            'tp': tp.loc[:, ['x', 'y', 'radius']].values,
            'fp': fp.loc[:, ['x', 'y', 'radius']].values,
            'fn': fn.loc[:, ['x', 'y', 'radius']].values}
        img_res_h2 = {'img_id': image_id, 'TP': len(tp), 'FP': len(fp), 'FN': len(fn),
            'time': t, 'normal': image_id in normal_ids}
        res_h2.append(img_res_h2)

    results_hough_h1 = pd.DataFrame(res_h1)
    results_hough_h2 = pd.DataFrame(res_h2)

    filename = f'hough_default'
    results_hough_h1.to_csv(results_path/f'{filename}_h1.csv')
    with open(results_path/f'{filename}_h1.pkl', 'wb') as f:
        pickle.dump(result_det_h1, f)

    results_hough_h2.to_csv(results_path/f'{filename}_h2.csv')
    with open(results_path/f'{filename}_h2.pkl', 'wb') as f:
        pickle.dump(result_det_h2, f)

100%|██████████| 118/118 [51:20<00:00, 26.11s/it] 


In [11]:
def plot_results(results_path):
    filename = 'md_th-0.95_mindist-6_area-14'
    results_morphology95 = pd.read_csv(results_path/f'{filename}.csv')
    results_morphology95['method'] = 'gs_morph_095'

    filename = 'md_th-0.97_mindist-6_area-14'
    results_morphology = pd.read_csv(results_path/f'{filename}.csv')
    results_morphology['method'] = 'gs_morph_097'

    filename = 'hdog_ms-1_Ms-3_ns-20_dth-0.006_dmd-6'
    results_hdog = pd.read_csv(results_path/f'{filename}.csv')
    results_hdog['method'] = 'hdog'

    results_hough_h1 = pd.read_csv(results_path/'hough_default_h1.csv')
    results_hough_h1['method'] = 'hough_h1'
    results_hough_h2 = pd.read_csv(results_path/'hough_default_h2.csv')
    results_hough_h2['method'] = 'hough_h2'

    results = pd.concat([results_morphology95, results_morphology, results_hdog, results_hough_h1, results_hough_h2], ignore_index=True)
    results['FPR'] = results['FP'] / (results['FP'] + results['TP'])
    results['mfpi'] = (results['FP'] / len(results)).sum()
    results['sens'] = results['TP'] / (results['TP'] + results['FN'])
    results.loc[~results.normal, 'fp_per_tp'] = results['FP'] / (results['TP'])
    results.loc[results.normal, 'fp_per_tp'] = results['FP']

    plt.figure(figsize=(10, 4))
    plt.title('Sensitivity vs method')
    sns.boxplot(data=results, x='method', y='sens')
    sns.despine()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.title('False positives per tp per image vs method')
    sns.boxplot(data=results, x='method', y='fp_per_tp')
    plt.ylim(0, 2000)
    sns.despine()
    plt.show()


    plt.figure(figsize=(10, 4))
    plt.title('False positives per tp per image vs method')
    sns.boxplot(data=results, x='method', y='fp_per_tp')
    plt.yscale('log')
    sns.despine()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.title('False positives per image on normal images vs method')
    sns.boxplot(data=results.loc[results.normal], x='method', y='fp_per_tp')
    sns.despine()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.title('Computation time per image vs method')
    sns.boxplot(data=results, x='method', y='time')
    # plt.yscale('log')
    sns.despine()
    plt.ylabel('time [s]')
    plt.ylim([0, 70])
    plt.show()


## Run experiments with pectoral supression

In [2]:
db = INBreast_Dataset(
    return_lesions_mask=True,
    level='image',
    max_lesion_diam_mm=None,
    extract_patches=False,
    partitions = ['validation'],
    lesion_types = ['calcification', 'cluster'],
    cropped_imgs = True,
    keep_just_images_of_lesion_type = False,
    use_muscle_mask=True,
    ignore_diameter_px = 15
)

In [ ]:
results_path = Path.cwd().parent.parent / 'data/comparisson_between_detectors_wo_pectoral/'
results_path.mkdir(exist_ok=True, parents=True)


# GSM
threshold = 0.97
min_distance = 6
area = 14
run_experiment_gsm(threshold, min_distance, area, results_path)

threshold = 0.95
min_distance = 6
area = 14
run_experiment_gsm(threshold, min_distance, area, results_path)


# HDOG
dog_parameters = {
    'min_sigma': 1,
    'max_sigma': 3,
    'n_scales': 20,
    'sigma_ratio': None,
    'dog_blob_th': 0.006,
    'dog_overlap': 0.2,
    'dog_min_dist': 6,
}

hessian_parameters = {
    'method': 'eigenval',
    'hessian_threshold': None,
    'hessian_th_divider': 300
}

run_experiment_hdog(dog_parameters, hessian_parameters, results_path)


# HOUGH
dehazing_params = {'omega': 0.9, 'window_size': 11, 'radius': 40, 'eps': 1e-5}

hough1_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                'param1': 300, 'param2': 8,  'minRadius': 2, 'maxRadius': 20}

hough2_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                'param1': 300, 'param2': 10,  'minRadius': 2, 'maxRadius': 20}

run_experiment_hough(dehazing_params, hough1_params, hough2_params, db, results_path)

# RESULTS
plot_results(results_path)

## Run experiments with pectoral supression

In [ ]:
db = INBreast_Dataset(
    return_lesions_mask=True,
    level='image',
    max_lesion_diam_mm=None,
    extract_patches=False,
    partitions = ['validation'],
    lesion_types = ['calcification', 'cluster'],
    cropped_imgs = True,
    keep_just_images_of_lesion_type = False,
    use_muscle_mask=False,
    ignore_diameter_px = 15
)

In [ ]:
results_path = Path.cwd().parent.parent / 'data/comparisson_between_detectors_w_pectoral/'
results_path.mkdir(exist_ok=True, parents=True)

# GSM
threshold = 0.97
min_distance = 6
area = 14
run_experiment_gsm(threshold, min_distance, area, results_path)

threshold = 0.95
min_distance = 6
area = 14
run_experiment_gsm(threshold, min_distance, area, results_path)


# HDOG
dog_parameters = {
    'min_sigma': 1,
    'max_sigma': 3,
    'n_scales': 20,
    'sigma_ratio': None,
    'dog_blob_th': 0.006,
    'dog_overlap': 0.2,
    'dog_min_dist': 6,
}

hessian_parameters = {
    'method': 'eigenval',
    'hessian_threshold': None,
    'hessian_th_divider': 300
}

run_experiment_hdog(dog_parameters, hessian_parameters, results_path)


# HOUGH
dehazing_params = {'omega': 0.9, 'window_size': 11, 'radius': 40, 'eps': 1e-5}

hough1_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                'param1': 300, 'param2': 8,  'minRadius': 2, 'maxRadius': 20}

hough2_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                'param1': 300, 'param2': 10,  'minRadius': 2, 'maxRadius': 20}

run_experiment_hough(dehazing_params, hough1_params, hough2_params, db, results_path)

# RESULTS
plot_results(results_path)